In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
import torchtext.experimental
import torchtext.experimental.vectors
from torchtext.experimental.datasets.raw.text_classification import RawTextIterableDataset
from torchtext.experimental.datasets.text_classification import TextClassificationDataset
from torchtext.experimental.functional import sequential_transforms, vocab_func, totensor

import collections
import random
import time

In [2]:
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

In [4]:
def get_train_valid_split(raw_train_data, split_ratio = 0.7):

    raw_train_data = list(raw_train_data)
        
    random.shuffle(raw_train_data)
        
    n_train_examples = int(len(raw_train_data) * split_ratio)
        
    train_data = raw_train_data[:n_train_examples]
    valid_data = raw_train_data[n_train_examples:]
    
    train_data = RawTextIterableDataset(train_data)
    valid_data = RawTextIterableDataset(valid_data)
    
    return train_data, valid_data

In [5]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [6]:
class Tokenizer:
    def __init__(self, tokenize_fn = 'basic_english', lower = True, max_length = None):
        
        self.tokenize_fn = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, s):
        
        tokens = self.tokenize_fn(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.max_length is not None:
            tokens = tokens[:self.max_length]
            
        return tokens

In [7]:
max_length = 500

tokenizer = Tokenizer(max_length = max_length)

In [8]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
    
    token_freqs = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freqs.update(tokens)
                
    vocab = torchtext.vocab.Vocab(token_freqs, **vocab_kwargs)
    
    return vocab

In [9]:
max_size = 25_000

vocab = build_vocab_from_data(raw_train_data, tokenizer, max_size = max_size)

In [10]:
def process_raw_data(raw_data, tokenizer, vocab):
    
    raw_data = [(label, text) for (label, text) in raw_data]

    text_transform = sequential_transforms(tokenizer.tokenize,
                                           vocab_func(vocab),
                                           totensor(dtype=torch.long))
    
    label_transform = sequential_transforms(totensor(dtype=torch.long))

    transforms = (label_transform, text_transform)

    dataset = TextClassificationDataset(raw_data,
                                        vocab,
                                        transforms)
    
    return dataset

In [11]:
train_data = process_raw_data(raw_train_data, tokenizer, vocab)
valid_data = process_raw_data(raw_valid_data, tokenizer, vocab)
test_data = process_raw_data(raw_test_data, tokenizer, vocab)

In [12]:
class Collator:
    def __init__(self, pad_idx):
        
        self.pad_idx = pad_idx
        
    def collate(self, batch):
        
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)
        
        lengths = torch.LongTensor([len(x) for x in text])

        text = nn.utils.rnn.pad_sequence(text, padding_value = self.pad_idx)
        
        return labels, text, lengths

In [13]:
pad_token = '<pad>'
pad_idx = vocab[pad_token]

collator = Collator(pad_idx)

In [14]:
batch_size = 256

train_iterator = torch.utils.data.DataLoader(train_data, 
                                             batch_size, 
                                             shuffle = True, 
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size, 
                                             shuffle = False, 
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size, 
                                            shuffle = False, 
                                            collate_fn = collator.collate)

In [15]:
class GRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.gru = nn.GRUCell(emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim, output_dim)

    def forward(self, text, lengths):

        # text = [seq len, batch size]
        # lengths = [batch size]

        embedded = self.embedding(text)

        # embedded = [seq len, batch size, emb dim]

        seq_len, batch_size, _ = embedded.shape
        hid_dim = self.gru.hidden_size
                
        hidden = torch.zeros(batch_size, hid_dim).to(embedded.device)
        
        for i in range(seq_len):
            x = embedded[i]
            hidden = self.gru(x, hidden)
        
        prediction = self.fc(hidden)

        # prediction = [batch size, output dim]

        return prediction

In [16]:
class GRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.gru = nn.GRU(emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim, output_dim)

    def forward(self, text, lengths):

        # text = [seq len, batch size]
        # lengths = [batch size]

        embedded = self.embedding(text)

        # embedded = [seq len, batch size, emb dim]

        output, hidden = self.gru(embedded)

        # output = [seq_len, batch size, n directions * hid dim]
        # hidden = [n layers * n directions, batch size, hid dim]

        prediction = self.fc(hidden.squeeze(0))

        # prediction = [batch size, output dim]

        return prediction 

In [17]:
class GRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.gru = nn.GRU(emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim, output_dim)

    def forward(self, text, lengths):

        # text = [seq len, batch size]
        # lengths = [batch size]

        embedded = self.embedding(text)

        # embedded = [seq len, batch size, emb dim]

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths, enforce_sorted = False)

        packed_output, hidden = self.gru(packed_embedded)

        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [seq_len, batch size, n directions * hid dim]
        # hidden = [n layers * n directions, batch size, hid dim]

        prediction = self.fc(hidden.squeeze(0))

        # prediction = [batch size, output dim]

        return prediction 

In [18]:
input_dim = len(vocab)
emb_dim = 100
hid_dim = 256
output_dim = 2

model = GRU(input_dim, emb_dim, hid_dim, output_dim, pad_idx)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [20]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,775,658 trainable parameters


In [21]:
for n, p in model.named_parameters():
    print(f'name: {n}, shape: {p.shape}')

name: embedding.weight, shape: torch.Size([25002, 100])
name: gru.weight_ih_l0, shape: torch.Size([768, 100])
name: gru.weight_hh_l0, shape: torch.Size([768, 256])
name: gru.bias_ih_l0, shape: torch.Size([768])
name: gru.bias_hh_l0, shape: torch.Size([768])
name: fc.weight, shape: torch.Size([2, 256])
name: fc.bias, shape: torch.Size([2])


In [22]:
def initialize_parameters(m):
    if isinstance(m, nn.Embedding):
        nn.init.uniform_(m.weight, -0.05, 0.05)
    elif isinstance(m, nn.GRU):
        for n, p in m.named_parameters():
            if 'weight_ih' in n:
                r, z, n = p.chunk(3)
                nn.init.xavier_uniform_(r)
                nn.init.xavier_uniform_(z)
                nn.init.xavier_uniform_(n)
            elif 'weight_hh' in n:
                r, z, n = p.chunk(3)
                nn.init.orthogonal_(r)
                nn.init.orthogonal_(z)
                nn.init.orthogonal_(n)
            elif 'bias' in n:
                r, z, n = p.chunk(3)
                nn.init.zeros_(r)
                nn.init.zeros_(z)
                nn.init.zeros_(n)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

In [23]:
model.apply(initialize_parameters)

GRU(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (gru): GRU(100, 256)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [24]:
glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)

In [25]:
def get_pretrained_embedding(initial_embedding, pretrained_vectors, vocab, unk_token):
    
    pretrained_embedding = torch.FloatTensor(initial_embedding.weight.clone()).detach()    
    pretrained_vocab = pretrained_vectors.vectors.get_stoi()
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        if token in pretrained_vocab:
            pretrained_vector = pretrained_vectors[token]
            pretrained_embedding[idx] = pretrained_vector
        else:
            unk_tokens.append(token)
        
    return pretrained_embedding, unk_tokens

In [26]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(model.embedding, glove, vocab, unk_token)

In [27]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[ 0.0098,  0.0150, -0.0099,  ...,  0.0211, -0.0092,  0.0027],
        [ 0.0347,  0.0276,  0.0468,  ..., -0.0315, -0.0472, -0.0326],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [28]:
model.embedding.weight.data[pad_idx] = torch.zeros(emb_dim)

In [29]:
model.embedding.weight.data

tensor([[ 0.0098,  0.0150, -0.0099,  ...,  0.0211, -0.0092,  0.0027],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [30]:
optimizer = optim.Adam(model.parameters())

In [31]:
criterion = nn.CrossEntropyLoss()

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
model = model.to(device)
criterion = criterion.to(device)

In [34]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [35]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text, lengths in iterator:
        
        labels = labels.to(device)
        text = text.to(device)

        optimizer.zero_grad()
        
        predictions = model(text, lengths)
        
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text, lengths in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text, lengths)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'gru-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.654 | Train Acc: 60.73%
	 Val. Loss: 0.584 |  Val. Acc: 68.87%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.423 | Train Acc: 80.73%
	 Val. Loss: 0.332 |  Val. Acc: 86.04%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.252 | Train Acc: 90.15%
	 Val. Loss: 0.285 |  Val. Acc: 88.63%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.186 | Train Acc: 93.05%
	 Val. Loss: 0.286 |  Val. Acc: 89.40%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.116 | Train Acc: 95.85%
	 Val. Loss: 0.307 |  Val. Acc: 89.56%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.065 | Train Acc: 97.90%
	 Val. Loss: 0.354 |  Val. Acc: 89.64%
Epoch: 07 | Epoch Time: 0m 8s
	Train Loss: 0.042 | Train Acc: 98.74%
	 Val. Loss: 0.403 |  Val. Acc: 89.35%
Epoch: 08 | Epoch Time: 0m 8s
	Train Loss: 0.020 | Train Acc: 99.47%
	 Val. Loss: 0.408 |  Val. Acc: 89.35%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.010 | Train Acc: 99.81%
	 Val. Loss: 0.505 |  Val. Acc: 88.53%
Epoch: 10 | Epoch Time: 0m 7

In [39]:
model.load_state_dict(torch.load('gru-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.290 | Test Acc: 87.93%


In [40]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    length = torch.LongTensor([len(tokens)]).to(device)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor, length)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [41]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.06520231813192368

In [42]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.8539475798606873

In [43]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.15590433776378632

In [44]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.3470574617385864